In [1]:
import torch
import torch.nn as nn
from torch import optim
from utils import BasicDataset, count_dice_coeff_torchvision
from torch.utils.data import DataLoader, random_split
from torchvision.models.segmentation import fcn_resnet50

In [2]:
net = fcn_resnet50(num_classes=13)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.0001)

In [3]:
dir_img = "archive"
dir_mask = "archive"
img_scale: float = 0.5
dataset = BasicDataset(dir_img, dir_mask, img_scale)
val_percent = 0.3

n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train_set, val_set = random_split(dataset, [n_train, n_val], generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_set, shuffle=True, batch_size=1)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, batch_size=10)

Creating dataset with 8080 examples


In [ ]:
for epoch in range(2): 
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)['out']
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i%100 == 0:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')
            running_loss = 0.0

In [6]:
torch.save(net.state_dict(), "model.pt")

In [ ]:
net = fcn_resnet50(num_classes=13)
net.load_state_dict(torch.load("model.pt"))
net.eval()
count_dice_coeff_torchvision(val_loader, net)